In [1]:
import numpy as np
import jax.random as jrandom
from jax import vmap, value_and_grad, jit
import matplotlib.pyplot as plt
import jax.numpy as jnp
import tqdm
import optax
import time

In [2]:
from QAG import data
from QAG import circuits as qag_circuits
from QAG.pennylane import qag

In [3]:
from zprop import propagator, circuits, loss

In [4]:
E_train, _ = data.make1D(path="/home/samonaco/Ele_FixedAngle", n_qubit=8, n_event=1000)

Original images:
   Number of events (training): 10000
   Image dimensions (training): (51, 51, 25)
   Size of each image: 0.52 MBs

Processing 1922 images at a time to fit process in 2 GBs of memory...

Downsampling Training set...


1017/1000  11532/10000: : 1017it [00:26, 38.88it/s]                       


In [12]:
n_iteration = 4
n_qubit = np.shape(E_train)[1]

In [13]:
cutoff1, cutoff2 = [5], [10]

qag_model = qag.Qag(E_train=E_train, n_iteration=n_iteration, circuit_block=qag_circuits.block_TREEDOWN)
for c1, c2 in zip(cutoff1, cutoff2):
    prop_model = propagator.Propagator(n_qubit = n_qubit, n_iteration = n_iteration, ansatz = circuits.id9, cutoff=c1, cutoff2=c2)
    f, _ = prop_model.Hlambdify()
    g = lambda p_a, p_p : f(p_p=p_p, p_a=p_a)
    v_g = jit(vmap(g, in_axes=(0, None), out_axes=0))
    qag_model.jv_q_circuit = v_g
    qag_model.train(n_epoch=500, n_image=300)

LOSS MMD: 0.4220 | LOSS COR: 0.6950:  98%|█████████▊| 488/500 [04:08<00:06,  1.96it/s]


KeyboardInterrupt: 